<font size="6">Улучшение сходимости нейросетей и борьба с переобучением</font>

# Нормализация входных данных


В 5й лекции мы убедились что нормализация данных оказывает заметное влияние на сходимость модели. Давайте разберемся почему так происходит.


1. Масштаб. Для табличных данных это было очевидно. И хотя пикселы изображения ближе по абсолютной величине друг к другу для них масштабирование тоже может быть актуально.

2. Стабильность. Малые изменения весов меньше влияют на результат. 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_stab.png" width="700">

3. Скорость обучения

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_speed.png" width="500">

4. Активации

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_activations.png" width="500">



Представим себе, что данные, которые мы подаем в нейросеть, распределены следующщим образом

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/no_normalization.png" width="300">


Фактически нейросети работают со скалярными произведениями. В этом плане две вектора, изображенных на рисунке, не сильно отличаются. Так же и точки нашего датасета слабо разделимы. Чтобы с этим работать, нейросеть сначала должна подобрать удобное преобразование, а затем только сравнивать наши объекты. Понятно, это усложняют задачу. 

Для того, чтобы облегчить нейросети задачу, входные признаки часто нормируют:

$$x1' = \dfrac {x1 - \mu_{x1}} {\sigma_{x1}}$$
$$x2' = \dfrac {x2 - \mu_{x2}} {\sigma_{x2}}$$

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/normalization.png" width="300">


 Такое преобразование действительно помогает нейросети 
 




###В чем состоит нормализация:

Вычитаем среднее, делеим на стандартное отклонение.

Для табличных данных иногда применяют PCA а затем снова нормализуют. Но для изображений такая техника не применяется (Почему?)

 <img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_pca.png" width="700">

Нормализация изображений в Pytorch 

In [ ]:
torchvision.transforms.ToTensor() # x /= 255 -> [0 .. 1]
# per channel mean, std
torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],[0.2470, 0.2434, 0.2615]) 
                                            

# Инициализация весов



То что нормализация входных данных полезна мы  убедились при работе на неглубоких моделях.

Но что будет если в модели несколько слоев?

Входом каждого следующего слоя буде выход предшествующего.

А его значения зависят как от входа так и от весов.

 <img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/weights_nn.png" width="700">

 Давайте посмотрим на распределение выходов внутренних словев.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (20,12)

def show_act(ax,x):
  ax.hist(x, bins=500,  range=(-3,3))
  std = x.std()
  ax.title.set_text(f"std {std:.9f}")

# Model from forth lecture with added third layer
class NeuralNet:
  def __init__(self):
    self.W1 = np.random.randn(3072, 500) * 0.01 
    self.W2 = np.random.randn(500, 100) * 0.01  # 0.1 10
    self.W3 = np.random.randn(100, 10) * 0.01 # 0.5 100
   

  def predict(self,x):

    fig, axs = plt.subplots(4, 1)
    fig.tight_layout()
    show_act(axs[0],x)

    # Layer 1
    scores1 = x.dot(self.W1) # Linear / Fully connected layer1
    activations1 = np.tanh(scores1) # ReLU activation
    show_act(axs[1],activations1)
    
    # Layer 2
    scores2 = activations1.dot(self.W2)
    activations2 = np.tanh(scores2) # ReLU activation
    show_act(axs[2],activations2)
    
    # Layer 3
    scores3 = activations2.dot(self.W3)
    show_act(axs[3],scores3)

    return scores3

x = np.random.randn(3072) # input image
simple_net = NeuralNet()
scores = simple_net.predict(x)


Выходы схлопаваются.


И это связанно с весами. Кстати почему мы вообще так их инициализируем.

Почему бы не использовать ноль ..

или 1- цу
...

или сделать их побольше, мы же все время на них умножаем, а выходы и так маленькие.

.. 

похоже что это работает если на каждом слое масштабировать веса, то коллапса можно избежать...


Хотим задать изначальные веса в нейросети. 
Как это можно сделать? 

Делать инициализацию 0 - плохая идея. В этом случае нейросеть получим, что градиент по всем таким весам будет одинаков - и их обновление, следовательно, тоже. Ничего не выучим. 

Можно инициализировать веса нормальным шумом с матожиданием 0 и маленькой дисперсией. Маленькая дисперсия нужна затем, чтобы не получить огромные градиенты за большие изначальные ошибки в предсказании. 

Но почему бы не взять другое распределение? Может, веса, инициализированные экспоненциальным распределениям дают какие-нибудь классные гарантии? Или какой-нибудь другой, не приходящий сразу в голову способ инициализации. Давайте это обсудим. 




Оказывается, иногда чтобы побороть такое поведение нейросети достаточно лишь правильно инициализировать веса 

Хотим задать изначальные веса в нейросети. 
Как это можно сделать? 

Делать инициализацию 0 - плохая идея. В этом случае получим, что градиент по всем таким весам будет одинаков - и их обновление, следовательно, тоже. Ничего не выучим. 

Можно инициализировать веса нормальным шумом с матожиданием 0 и маленькой дисперсией. Маленькая дисперсия нужна затем, чтобы не получить огромные градиенты за большие изначальные ошибки в предсказании. 

Но почему бы не взять другое распределение? Может, веса, инициализированные экспоненциальным распределениям, дают какие-нибудь классные гарантии? Или какой-нибудь другой, не приходящий сразу в голову способ инициализации. Давайте это обсудим. 


### Инициализация Xavier

Задача: сохранить распределение активаций

$$D(in) = D(out)$$

  $in = z_i$ - вход i-того слоя

  $ w_i $- веса i-того  слоя

  $ out = z_iw_i $ - выход i-того слоя (активации)


$$D(z_i) = D(z_iW_i)$$

##### $\color{brown}{\text{Вывод Xavier }}$





Рассмотрим в качестве активации нечетную функцию с единичной производной в нуле.

Например, нам подойдет гиперболический тангенс (tanh)

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)

x = np.arange(-10, 10.1, 0.1)
plt.plot(x, np.tanh(x))
plt.show()

Мы хотим начать из линейного региона этой функции, чтобы избежать
затухающих градиентов

Как у нас зависят активации на текущем слое от активаций на предыдущем?

$$z^{i+1} = f(z^iW^i)$$

Тогда, так как мы вначале хотим находиться в районе линейности нашей функции, то

$$z^{i+1} \approx z^i W^i$$

Ответить на вопрос, а как будут связаны дисперсии этих активаций, сложнее 

Сначала распишем:
1. чему равна дисперсия суммы двух независимых величин

$$D(\eta + \gamma) = D\eta + D\gamma$$

2. чему равна дисперсия произведений двух независимых величин


$$D\eta\gamma = E(\eta\gamma)^2 - (E\eta\gamma)^2 = E\eta^2E\gamma^2 - (E\eta)^2(E\gamma)^2$$ 

Далее распишем для одного веса текущего слоя 

$$z^{i+1}_{k} = \sum_t z^i_t w_{kt}$$

$$D(z^{i+1}_{k}) = D(\sum_t z^i_t w_{kt}) = \sum_t D(z^i_t w_{kt})$$

Предполагая, что дисперсии весов и активаций одинаковые (а нам бы так хотелось)

$$D(z^{i+1}_{k}) = n D(z^i_0 w_{k0})$$

Далее применяем нашу формулу и получаем:

$$D(z^{i+1}_{k}) = n [E(z^i_0)^2E(w_{k0})^2 - (Ez^i_0)^2(Ew_{k0})^2]$$

Хотим, чтобы матожидание наших активаций оставалось равным нулю, и мы можем этого добиться, делая  матожидание весов равным 0. 

$$D(z^{i+1}_{k}) =   n E(z^i_0)^2E(w_{k0})^2 $$

Заметим, что, так как матожидание активаций и весов равны 0, то матожидания их квадратов равны дисперсии активаций и весов соответственно:

$$D(z^{i}_{0}) = E(z^{i}_{0})^2 - (Ez^{i}_{0})^2 = E(z^{i}_{0})^2$$
$$D(w_{k0}) = E(w_{k0})^2 - (Ew_{k0})^2 = E(w_{k0})^2$$

Подставив их в выражение для $D(z^{i+1}_{k})$ получим:

$$D(z^{i+1}_{k}) = n D(z^i_0)D(w_{k0})$$


Отсюда можно вывести формулу для зависимости активаций любого слоя от весов предыдущих слоев и дисперсии исходных данных

$$Dz^i = Dx \prod_{p=0}^{i-1}n_pDW^p $$

Где $n_p$ - размерность выхода слоя p-го слоя

Аналогично можно вывести формулу для градиентов по активациям


$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}n_{p+1}DW^p $$



Для хорошей инициализации нам бы хотелось, чтобы дисперсии активаций на всех слоях были одинаковые. 
А еще хотелось бы, чтобы дисперсии градиентов тоже были одинаковые. 

Тогда у нас не происходит резких скачков в распределении активаций, а градиент не затухает и не взрывается 


$$Dz^i = Dz^j$$
$$D\dfrac {\delta L} {\delta z^i} = D\dfrac {\delta L} {\delta z^j}$$


Учитывая предыдущее, это эквивалентно тому, что мы требуем

$$n_iDW^i = 1$$
$$n_{i+1}DW^i = 1$$

Одновременно так сделать не получится $$n_i \ne n_{i+1}$$

Потому делаем компромисс - среднее гармонческое решений первого и второго уравнения

$$DW^i = \dfrac 2 {n_i + n_{i+1}}$$

Надо выбрать распределение.

$$ EW^i = 0 $$

$$DW^i = \dfrac 2 {n_i + n_{i+1}}$$

Можно было бы взять нормальное распределение с такими параметрами.

$$W_i \sim N(0, sd=\sqrt{\dfrac 2 {n_i + n_{i+1}}}) $$

А можно равномерное:

$$D(U[a, b]) = \dfrac 1 {12} (b -a)^2$$

Итого получим:
$$W_i \sim U[-\dfrac {\sqrt{6}} {n_i + n_{i + 1}}, \dfrac {\sqrt{6}} {n_i + n_{i + 1}} ]$$


#### Формула

$n_i$ - количество входов слоя*

$$ D(W_i) = 1/n_i $$

Аналогично можно рассуждать по поводу градиентов тогда: 

$$ D(W_i) = \dfrac {2} {n_i + n_{i +1}} $$

Дисперсия весов зависит только от количества входов слоя

* *Для линейного слоя количество входов == количеству весов*

Проверим:

$std = \sqrt {Variance}$

In [ ]:
from math import sqrt

plt.rcParams["figure.figsize"] = (20,12)

class NeuralNetXavier(NeuralNet):
  def __init__(self):
    # std = sqrt(variance)
    self.W1 = np.random.randn(3072, 500) * sqrt(1 / 3072)
    self.W2 = np.random.randn(500, 100) * sqrt(1 / 500)  
    self.W3 = np.random.randn(100, 10) * sqrt(1 / 100) 

xavier_net = NeuralNetXavier()
scores = xavier_net.predict(x)

## He-инициализация (Kaiming)

Но многие люди используют ReLU. Она зануляет половину весов

$$ D(W_i) = 2*(1/n_i) = 2/n_i $$

##### $\color{brown}{\text{Вывод Kaiming }}$



Тогда получатся похожие условия 

$$Dz^i = Dx \prod_{p=0}^{i-1}\dfrac 1 2 n_pDW^p $$

$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}\dfrac 1 2 n_{p+1}DW^p $$


И для них решения будут тоже похожими:

$$\frac 2 {n_k}$$

и

$$\frac 2 {n_{k+1}}$$

Можно опять взять среднее гармоническое. Но на практике, особенно в случае сверточных нейронных сетей, просто берут либо $ \frac 2 {n_i}$ либо $\frac 2 {n_i + 1}$


Итого получим:

$$W^i \sim N(0, sd=\sqrt{\frac 2 n_i})$$

$$W^i \sim N(0, sd=\sqrt{\frac 2 {n_i + 1}})$$

Опять же, можно использовать и равномерное распределение

### Сверточный слой

Для сверточных слоев

$n = Channels_num * Kernel_{Height} * Kernel_{Width}$

## Важность инициализации весов

1. Нейросеть может сойтись значительно быстрее


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/weight_initialization_influence_convergence_neural_networks.png" width="550">

2. В зависимости от выбранной активации сеть вообще может сойтись или не сойтись

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/activation_function_influence_convergence_neural_networks.png" width="550">

## Обобщение инициализаций Ксавьера и He-инициализации

Вообще говоря, коэффициенты в инициализациях (числитель в формуле для дисперсии), зависят от конкретной выбранной функции активации.
[В pytorch есть функции для вычисления этих коэффициентов](https://pytorch.org/docs/stable/nn.init.html)


## Ортогональная инициализация

Также иногда используется так называемая ортогональная инициализация весов. 
Для каждого слоя мы убеждаемся, что изначальная матрица весов является ортогональной. 

Выберем ортогональную матрицу весов 
$$W: WW^T = 1$$

Тогда:
1.  норма активации сохраняется (опять же, активации между слоями остаются в одном масштабе)
$$||s_{i+1}|| = ||W_{i}s_i|| = ||s_i||$$

2.  все нейроны делают «разные» преобразования
$$ ⟨W_i, W_j⟩ = 0~i \ne j$$
$$ ⟨W_i, W_j⟩ = 1~i = j$$


Иногда такая инициализация обеспечивает значительно лучшую сходимость, [тут можно почитать об этом подробнее](https://datascience.stackexchange.com/questions/64899/why-is-orthogonal-weights-initialization-so-important-for-ppo) 

## Инициализация весов в Pytorch

Для инициализации весов Pytorch используется модуль torch.nn.init

В нем определены разные функции для инициализации весов.

Нюанс состоит в том, что обычно для слоев разного типа может требоваться разная регуляриация. Потому в функции, которая инициализирует слов вашей нейронное сети желательно прописывать разное поведение для разных слоев.

Попробуем, например, добавить в нашу нейросеть инициализацию. Нам нужна регуляризация Xavier, так как у нас Sigmoid

In [ ]:
import torch
import torch.nn as nn

class LeNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.activation = nn.ReLU()
        self.pipeline = nn.Sequential(
          nn.Conv2d(1, 6, 5),
          self.activation,
          nn.Conv2d(6, 16, 5),
          self.activation,
          nn.Linear(16 * 4 * 4, 120),  # 5*5 from image dimension
          self.activation,
          nn.Linear(120, 84),
          self.activation,
          nn.Linear(84, 10),
        )

    def forward(self, x):
        return self.pipeline(x)


In [ ]:
model = LeNet()

with torch.no_grad():
  for name, module in model.named_modules():
    if type(module) in  (nn.Linear,nn.Conv2d):
      print(f"Set weighs in module {name}")
      torch.nn.init.kaiming_normal_(module.weight,  nonlinearity='relu')


Альтернативный способ

In [ ]:
##https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
def init_weights(module):
  with torch.no_grad():
    if type(module) in  (nn.Linear,nn.Conv2d):
      print("set")
      torch.nn.init.kaiming_normal_(module.weight,  nonlinearity='relu')

model.apply(init_weights)

# Затухание градиента

Откуда оно берется? 

Посмотрим на обычную сигмоиду

$$\sigma(z) = \dfrac 1 {1 + e^{-z}}$$

Ее производная, как вы уже выводили, равна

$$\dfrac {\delta \sigma(z)} {\delta z} = \sigma(z) (1 - \sigma(z))$$

Какое максимальное значение у такой функции?

Учтем, что область значений  сигмоида от 0 до 1

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn-whitegrid')

x = np.arange(0, 1.001, 0.001)
plt.plot(x, x - x**2)
plt.title('Derivative of the sigmoidal function', size = 15)
plt.show()

Получается, что максимальное значение производной по сигмоиде - 1/4

Теперь возьмем простую нейронную сеть 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/simple_nn_with_sigmoid.png" width="750">

Посчитаем у нее градиент

$$\dfrac {\delta L} {\delta z_4} = \dfrac {\delta L} {\delta y} \dfrac {\delta y} {\delta z_4} = \dfrac {\delta L} {\delta y} \dfrac {\delta \sigma(w_5z)} {\delta z} w_5 \le \dfrac 1 4 \dfrac {\delta L} {\delta y}  w_5 $$

Аналогично можно посчитать градиент для $z_3$

$$\dfrac {\delta L} {\delta z_3} = \dfrac {\delta L} {\delta z_4} \dfrac {\delta z_4} {\delta z_3} \le \dfrac {\delta L} {\delta y} \dfrac {\delta \sigma(w_4z)} {\delta z} w_5 \le {\dfrac 1 4}^2 \dfrac {\delta L} {\delta y}  w_5 w_4$$

И так далее

$$\dfrac {\delta L} {\delta x}  \le {\dfrac 1 4}^5 \dfrac {\delta L} {\delta y}  w_5 w_4 w_3 w_2 w_1$$

Таким образом, градиент начинает экспоненциально затухать, если веса маленькие.

Если веса большие - то градиент наоборот начнет экспоненциально возрастать - все взорвется

Для некоторых функций активации картина будет не столь катастрофична, но тоже неприятна.

В семинарском задании вы посмотрите, например, как ведет себя функция ReLU в этом случае.

### Плохой вариант борьбы с этим

Давайте на каждом слое просто нормировать каждый признак, используя среднее и дисперсию по батчу

$$ \hat{x}_{i} = \frac{x_{i} - \mu_{B}}{\sigma_{B} + \epsilon}$$




Проблема в том, что таким образом мы можем попасть в область линейной составляющей нашей функции. Например, в случае сигмоиды 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/domain_of_linear_of_sigmoid_function.png" width="500">

Получаем набор линейных слоев фактически без функций активации -> все вырождается в однослойную сеть. Не то, что нам надо

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/bn_relu.png" width="400">

В случае с ReLU подобной функцией активации, проблемы не исчезат. Если нормировать активации в интервал [0,1] то на каждом слое мы будем всегда обнулять половину активаций. 

Хотя опыт принудительной нормализации выходов слоя известен: в AlexNet было два слоя нормализации.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/bn_loss.png" width="600">

В начале обучения Loss сильно меняется. В связи с этим происходят большие обновления весов в результате чего значтельная часть активаций может занулиться или уйти в область где Relu линейна.

На этом этапе нормализация могла бы помочь. Но по мере обучения сети изменения весов становятся более осмысленными. И веса каждого слоя подстраиваются под свое распределение данных.




## BatchNormalization

Нам надо дать нейронной сети возможность перемещать распределение выходов слоя из области 0 и самой подбирать дисперсию. Для этой цели используется __батч-нормировка__ (_batch normalization_), которая вводит в нейронную сеть дополнительную операцию между соседними скрытыми слоями. Она состоит из нормализации входящих (в слой батч-нормировки) значений, полученных от одного скрытого слоя, масштабирования и сдвига с применением двух новых параметров и передачи полученных значений на вход следующему скрытому слою.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img_license/nn_example.png" width="800">

Параметры, используемые в батч-нормировке $\gamma$ &mdash; отвечающий за сжатие и $\beta$ &mdash; отвечающий за сдвиг являются обучаемыми параметрами (наподобие весов и смещений скрытых слоев). 

Помимо обучаемых параметров $\gamma$ и $\beta$ в слое батч-нормировки существуют также необучаемые параметры: __скользящее среднее__ средних (_Mean Moving Average_) и скользящее среднее дисперсий (_Variance Moving Average_), служащие для сохранения состояния слоя батч-нормировки. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/batchnorm_layer_parameters.png" >  

Параметры $\gamma$, $\beta$, а также оба скользящих средних вычисляются для каждого слоя батч-нормировки отдельно и являются векторами с длиной, равной количеству входящих признаков.

В процессе обучения, мы подаем в нейронную сеть по одному мини-батчу за раз. Процедуру обработки значений $x$ из одного мини-батча $ B = \{x_{1},\ldots, x_{m}\} $ можно представить следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/batch_normalization_compute_moving_average.png" width="1000">

Шаг масштабирования и сдвига является главным новшеством батч-нормировки, поскольку в отличие от предыдущего рассмотренного примера, нормированные значения больше не обязаны иметь среднее равное 0 и единичную дисперсию. Батч-нормировка позволяет сдвигать среднее нормированных значений и масштабировать дисперсию. Фактически, теперь нейросеть даже может отменить нормализацию, если считает ее ненужной.

Для наглядности проиллюстрируем размерности промежуточных переменных на следующем изображении:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/batch_normalization_compute_moving_average_scheme.png" width="1000">

После прямого прохода, параметры $\gamma$ и $\beta$ обновляются через обратное распространение ошибки также как и веса скрытых слоев.

### Скользящее среднее

Выше мы обсуждали то, что в процессе обучения слой  Батч-нормировки рассчитывает значение среднего и дисперсии каждого признака в соответствующем мини-батче. Однако во время предсказания, батч у нас уже отсутствует &mdash; откуда брать среднее и дисперсию? 

Во время предсказания используется скользящее среднее, которое было рассчитано и сохранено в процессе обучения. 

$$ \mu_{mov_{B}} = (1-\alpha)\mu_{mov_{B-1}}+\alpha\mu_{B} $$

$$ \sigma_{mov_{B}} = (1-\alpha)\sigma_{mov_{B-1}}+\alpha\sigma_{B} $$

Обычно используется параметр $\alpha = 0.1$

Почему используется именно скользящее среднее, а не, например, статистика всей обучающей выборки? Дело в том, что при таком подходе нам бы пришлось хранить средние всех признаков для всех батчей, пропущенных через нейросеть в ходе обучения. Это ужасно невыгодно по памяти. Вместо этого скользящее среднее выступает в качестве приближенной оценки среднего и дисперсии обучающего набора, в этом случае эффективность использования ресурсов увеличивается, поскольку вычисления производятся постепенно &mdash; нам нужно хранить в памяти только одно число &mdash; значение скользящего среднего, полученное на последнем шаге.

Проиллюстрировать преимущество использования скользящего среднего можно на следующем примере:

Предположим, что у нас есть некоторая генеральная совокупность объектов, обладающих некоторым признаком $x$, (соответствующая абстрактной обучающей выборке) и некоторый черный ящик, извлекающий по $k$ объектов из этой генеральной совокупности (соответствующий абстрактному даталоадеру). Наша задача &mdash; дать оценку ожидаемому среднему этих $k$ объектов. При этом ожидаемое среднее для $k$ объектов может отличаться от среднего генеральной совокупности, поскольку могут накладываться дополнительные условия на извлекаемую выборку. В данном примере для простоты будем извлекать $k$ объектов из некого распределения случайным образом:

In [ ]:
import sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

k = 500 # sample size
n = 2
p = 0.5

sample = np.random.negative_binomial(n, p, k)
sns.histplot(data=sample, discrete=True)
plt.show()

Оценить ожидаемое среднее теоретически, не зная как распределен признак $x$ наших объектов, трудно. Мы можем собрать большое количество средних и произвести оценку с их помощью, но для этого нам потребуется хранить в памяти все эти значения, что опять-таки приведет к неэффективному расходу ресурсов. Более эффективным решением будет воспользоваться скользящим средним. Давайте сравним эти два метода:

In [ ]:
ema = 0
alpha = 0.01
means = np.array([])

for i in range(10000):
    sample = np.random.negative_binomial(n, p, 50)
    ema = (1 - alpha) * ema + alpha * sample.mean()
    means = np.append(means, sample.mean())

Посчитаем количество памяти, затрачиваемое на хранение списка средних значений признака $x$ по выборкам из $k$ объектов, и количество памяти, затрачиваемое на хранение скользящего среднего:

In [ ]:
print(f'Количество памяти для хранения скользящего среднего:\t{sys.getsizeof(ema)} байт')
print(f'Количество памяти для хранения списка средних:\t{sys.getsizeof(means)} байт')

Видно, что на хранение массива средних значений расходуется на порядки больше памяти, чем на хранение одного скользящего среднего. Теперь давайте воспользуемся тем, что мы сэмплировали случайные выборки из известного распределения, и можем теоретически рассчитать их среднее. В нашем примере мы извлекали выборки из негативного биномиального распределения с параметрами $n=2$ и $p=0.5$, для которого среднее рассчитывается по формуле $mean=\frac{np}{1-p}=2$. Мы знаем, что при достаточно большом количестве сэмплированных выборок, среднее распределения выборочных средних будет стремиться к среднему генеральной совокупности. Сравним результаты, полученные с использованием сохраненных выборочных средних и скользящего среднего с теоретическим расчетом:

In [ ]:
print(f'Среднее признака x по k объектам, оцененное с помощью скользящего среднего:\t{ema:.8f}')
print(f'Среднее признака x по k объектам, оцененное по всем сэмплированным выборкам:\t{means.mean():.8f}')

Видно, что мы получили довольно точную оценку, использовав скользящее среднее.

### Защита от нулей в знаменателе

Чтобы у нас не мог возникнуть 0 в знаменателе, добавляем маленькое число - $\epsilon$. Например, равное 1e-5


$$ \hat{x}_{i} = \frac{x_{i} - \mu_{B}}{\sigma_{B} + \epsilon}$$

$$ BN_{\gamma, \beta}(x_{i}) = \gamma \hat{x}_{i} + \beta $$

Pytorch

In [ ]:
import torch 
from torch import nn


dummy = torch.randn((64,784)) #batch_size , n_features
bn_layer = torch.nn.BatchNorm1d(784) # n_features
bn_layer(dummy)

print("Input",dummy.shape)
print("Gamma", bn_layer.weight.shape)
print("Betta", bn_layer.bias.shape)




### BN для Сверточного слоя

Слой конволюции можно свести к линейному слою с очень жесткими ограничениями на веса. Поэтому неудивительно, что BatchNorm можно применять и для линейных слоев и для конволюций. 

С конволюциями есть единственный нюанс - у нас "одним признаком" считается вся получаемая **feature map**. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/feature_map.png" width="500">

И нормализация идет по всей такой feature map (по всему каналу) для всех объектов. 

In [ ]:
import torch 
from torch import nn


dummy = torch.randn((64,3,32,32)) # batch_size , channels, W, H
bn_layer = torch.nn.BatchNorm2d(3) # channels
bn_layer(dummy)

print("Input",dummy.shape)
print("Gamma", bn_layer.weight.shape)
print("Betta", bn_layer.bias.shape)

### Пример работы

Этот метод действительно работает. 
Видим, что нейросети с батчнормализацией:

1. Сходятся быстрее, чем нейросети без 
2. Могут работать с более высоким начальным learning rate, причем это позволяет достигать лучших результатов
3. BatchNorm позволяет глубокой нейросетке работать даже с функцией активации в виде сигмоиды. Без батчнорма такая сеть не обучилась бы вовсе. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_efficiency.png" width="550">


##### $\color{brown}{\text{ Градиент }}$

Вычисление градиента batchnorm - интересное упражнение на понимание того, как работает backpropagation. В лекции мы это опускаем, можете ознакомиться самостоятельно

[Deriving the Gradient for the Backward Pass of Batch Normalization](https://kevinzakka.github.io/2016/09/14/batch_normalization/)



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_gradient.png" width="700">

### Советы по использованию BatchNormalization

Стоит помнить, что с батч-нормализацией:

* **Крайне важно** перемешивать объекты (составлять новые батчи) между эпохами. Единицей обучения параметров $\beta$ и $\gamma$ являются батчи. Если их не перемешивать, то из 6400 объектов в тренировочном датасете получим лишь 100 объектов (при условии, что в батче 64 объекта) для обучения $\beta$ и $\gamma$

* В слое, после которого поставили BatchNormalization, надо убрать смещения (параметр $\beta$ в BatchNormalization берет эту роль сам по себе)


* Другое расписание learning rate: бОльшее значение в начале обучения и быстрое уменьшение в процессе обучения

* Если используем BatchNormalization, то надо уменьшить силу Dropout и L2-регуляризации

* Чем меньше размер батча в обучении, тем хуже будет работать BatchNormalization

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnorm_batch_size.png" width="550">



### Используем BatchNormalization в Pytorch

In [ ]:
class SimpleMNIST_NN_Init_Batchnorm(nn.Module):
    def __init__(self, n_layers):
        super().__init__()
        self.n_layers = n_layers
        layers = [nn.Linear(28 * 28, 100, bias=False),
                  nn.BatchNorm1d(100),
                  nn.Sigmoid()]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100, bias=False))
            layers.append(nn.BatchNorm1d(100))
            layers.append(nn.Sigmoid())
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        return x

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                torch.nn.init.xavier_normal_(child.weight,
                                             gain=sigmoid_gain)
                if child.bias is not None:
                    torch.nn.init.zeros_(child.bias)

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)
writer = get_summary_writer("batchnorm2")
register_model_hooks(model, writer)
train_model_sep(model, writer, optimizer)

И попробуем, согласно советам, увеличить learning rate

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
writer = get_summary_writer("batchnorm_increased_lr")
register_model_hooks(model, writer)
train_model_sep(model, writer, optimizer)

### Ставить BatchNormalization до или после активации?



#### До


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/batchnormalization_before_activation.png" width="400">

* Рекомендуется авторами статьи, где предложили Batch Normalization
* Для сигмоиды, BN, поставленная после активации, не решает проблем сигмоиды

#### После

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/batchnormalization_after_activation.png" width="400">

* Аргументация авторов статьи не до конца обоснована
* Обычно, сигмоиду не используют в современных нейронных сетях
* Для популярной ReLU, BN, поставленная до активации может приводить к “умирающей ReLU”, когда большая часть ее входов меньше 0 и потому для них градиент не проходит
* На многих задачах BN после функции активации работает лучше или не хуже поставленной до

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/batchnormalization_before_or_after_relu.png" width="500">

[BN experiments](https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md)

### Ставить BatchNormalization до или после Dropout?



#### До

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/batchnormalization_before_dropout.png" width="600">

* Меньше влияние (covariate shift) Dropout на Batchnorm

#### После

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/batchnormalization_after_dropout.png" width="600">

* Информация о зануленных активациях не просачивается через среднее и дисперсию батча

### Ставить только что-то одно 

* Dropout может отрицательно влиять на качество нейросети с BatchNorm за счет разного поведения на train и test




### Строго говоря

* Оптимальный порядок следования слоев зависит от задачи и архитектуры сети
* Возможно, стоит применять модифицированные версии BatchNorm

## Другие Normalization

Существует большое количество иных нормализаций, их неполный список можно найти [здесь](https://paperswithcode.com/methods/category/normalization). В данной секции мы рассмотрим самые популярные виды нормализации помимо BatchNorm.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/normalization_methods.png" width="900">

Чаще всего нормализации тестируют на изображениях, в том числе или только на изображениях. Потому их графически объясняют при помощи следующей схемы:

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/dimensions_channels_batch_samples.png" width="450">

*По оси абсцисс* расположены объекты из батча,  
*по оси ординат* - feature map, преобразованный в вектор,  
*по оси аппликат* - каналы (feature maps).

В этом представлении BatchNorm выглядит следующим образом:

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img_license/batch_norm_3d.png" width="450">

[Batch Normalization](https://paperswithcode.com/method/batch-normalization)

### Layer Norm

Помимо свёрточных нейронных сетей, существует специальный тип нейронных сетей, используемых для обработки последовательностей. Называется он "рекуррентные нейронные сети", ему же будет посвящена наша следующая лекция. 

Когда оказалось, что BatchNorm положительно сказывается на обучении нейронных сетей, его попытались применить для различных архитектур. BatchNorm нельзя было использовать "из коробки" для рекуррентных нейронных сетей (работающих с последовательными данными), пришлось придумывать различные адаптации, среди которых наиболее удачной оказалась **Layer Normalization**.  
По сути, теперь нормализация происходит внутри одного объекта из батча, а не поканально в рамках батча. С математической точки зрения, данная "адаптация" отличается от BatchNorm, однако экспериментально она превзошла своих конкурентов в задаче нормализации при обработке последовательных данных.

Впоследствии, данный метод нормализации хорошо проявил себя в трансформерах - наследниках рекуррентных нейронных сетей в вопросах обработки последовательных данных (об этом типе нейросетей мы также поговорим на следующей лекции). После успешного применения трансформеров в задачах компьютерного зрения, LayerNorm стал использоваться и в компьютерном зрении (хотя и уступал BatchNorm при использовании в свёрточных нейронных сетях).

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img_license/layer_norm_3d.png" width="450">

[Layer Normalization](https://paperswithcode.com/method/layer-normalization)


### Instance Norm

Следующий вид нормализации был предложен отечественными исследователями (из СколТеха), занимавшимся разработкой быстрого и эффективного способа переноса стиля с одного изображения на другое. 

При использовании BatchNorm терялась информация о контрастах на конкретном изображении, поскольку нормализация производится по нескольким объектам. Для сохранения *контрастов* в экземпляре (*instance*) изображения была предложена специальная нормализация, рассматривающая конкретный канал одного конкретного объекта. Было предложено два названия нормализации: связанное с мотивацией (contrast normalization) и связанное с принципом работы (**instance normalization**). 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img_license/instance_norm_3d.png" width="450">

[Instance Normalization](https://paperswithcode.com/method/instance-normalization)

### Group Norm

В течение долгого времени, BatchNorm оставался однозначным фаворитом для использования в задачах компьютерного зрения, однако:
1. В связи с необходимостью точно считать статистики внутри batch, при обучении приходилось использовать большой batch size.  

2. Ограниченность размера видеопамяти вынуждает разработчиков идти на компромисс между сложностью модели и batch size.

Итого, использование BatchNorm приводило к невозможности использовать сложные модели**\***, поскольку им просто не хватало места на видеокарте. 

Необходимость использовать большой batch size могут решать различные нормализации, не использующие batch-размерность. К примеру, уже известные нам **Layer Norm** и **Instance Norm**. Эмпирически оказалось, что данные нормализации уступают BatchNorm по качеству работы: в то время как LayerNorm предполагает одинаковую важность и суть различных каналов (*рассматривая данные излишне глобально*), InstNorm упускает межканальные взаимодействия (*рассматривая данные слишком локально*). 

Успешным обобщением данных методов является **Group Normalization**: данный метод разбивает каналы на $G \in [1; C]$ групп, присваивая каждой из них (примерно) равное количество каналов. Отметим, что при $G = 1$, GroupNorm идентичен LayerNorm, а при $G = C$, GroupNorm идентичен InstNorm. 

Эмпирически оказалось, что при замене BatchNorm на GroupNorm качество модели падает в разы менее значительно, чем при использовании LayerNorm либо GroupNorm. Более того, при изменении batch size качество работы LayerNorm не изменялось, что открыло перспективы для создания более сложных моделей компьютерного зрения. 


**\*** - подразумевается, что уменьшение batch size позволило бы создать более сложные модели.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img_license/group_normalization_3d.png" width="450">

[Group Normalization](https://paperswithcode.com/method/group-normalization)

### Weight Normalization

Существует семейство методов, принципиально отличающихся от рассмотренных ранее нормализаций: в отличие от BatchNorm, LayerNorm и прочих известных вам нормализаций, ориентированных на **изменение представлений данных** внутри нейронной сети (и по сути представляемых как обычные слои), некоторые методы повышения качества обучения нейронных сетей ориентированы на **изменение параметров** самой нейронной сети.

[Weight Normalization](https://paperswithcode.com/method/weight-normalization) - один из таких методов, во многом вдохновлённый BatchNorm. В отличие от ранее упомянутых нормализаций, он появился в результате анализа влияния BatchNorm на обучение с сугубо математической точки зрения (было рассмотрено влияние BatchNorm на [информационную матрицу Фишера](https://en.wikipedia.org/wiki/Fisher_information)).  



В данном методе производится репараметризация слоя нейронной сети следующим образом:

$$\mathbb{w} = \frac{g}{\left\Vert \mathbb{v}\right\Vert}\mathbb{v},$$

где $\mathbb{w}$ - вектор весов слоя, $g$ и $\mathbb{v}$ - обучаемые параметры. Важный момент, который стоит сразу отметить: данный вид параметризации не зависит от данных, потому одинаково хорошо будет работать при любом batch size. 

**Weight normalization** в некоторых случаях позволяет с лёгкостью выполнять ранее обсуждаемую "умную инициализацию" параметров нейронной сети. Выполняется она следующим образом: некоторым образом инициализируем $\mathbb{v}$; выполним запуск сети на случайном batch, пусть на вход слою приходит $\mathbb{x}$. Выполним следующую операцию:

$$t = \frac{\mathbb{v}\cdot\mathbb{x}}{\left\Vert\mathbb{x}\right\Vert}, \qquad y = \phi\left(\frac{t - \mu[t]}{\sigma[t]}\right)$$

Чтобы перед активацией получить значения с нулевым средним и единичным стандартным отклонением, стоит установить параметрам $g$ и $b$ следующие значения:

$$g \leftarrow \frac{1}{\sigma[t]}, \qquad b \leftarrow \frac{-\mu[t]}{\sigma[t]}.$$

Другой важной особенностью **weight normalization** является *разделение* вектора весов на "направление вектора" и "норму вектора". По сути, отделение нормы векторов весов позволяет реализовывать гибкую настройку скорости обучения - различные параметры начинают обновляться с различной скоростью ($\sim\lambda \cdot g_i$), что позволяет модели более гибко обучаться. 

Weight normalization - далеко не единственный метод, позволяющий улучшать обучение модели, влияя на обновление её параметров: существует целое семейство "оптимизаторов" - методов, отвечающих за изменение параметров обучаемой модели.

# Регуляризация

Второй способ улучшения сходимости нейросетей и борьбы с переобучением - введение регуляризации. Ее можно вводить несколькими способами. 

## L1, L2 регуляризации

Самый простой способ мы уже разобрали - давайте просто добавим в лосс штраф к весам. На сходимость нейросети, это, правда, повлияет слабо

$$Loss\_reg = loss + \lambda \cdot reg$$

$$ reg_{L1} = \lambda \sum |w_i| $$

$$ reg_{L2} = \lambda \sum w_i^2 $$

<img src="https://edunet.kea.su/repo/EduNet-content/L07/img_license/l1_l2_regularization_to_weight.gif" alt="alttext" width="500px"/>

[Visualizing regularization and the L1 and L2 norms](https://towardsdatascience.com/visualizing-regularization-and-the-l1-and-l2-norms-d962aa769932)

Иногда уже его хватает, чтобы решить все проблемы. Напомним, что L2 лосс приводит к большому числу маленьких ненулевых весов в сети. А L1 лосс - к маленькому числу ненулевых весов (разреженной нейросети)



Классы из пакета torch.optim поддерживают L2 регуляризацию 

In [ ]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr =0.01, weight_decay = 2.5e-4) #weight_decay == lambda


## Batchnorm как регуляризация

Почему для нейросети с батчнормализацией можно использовать более высокие learning rate? 

 

Оказывается, батчнормализация делает неявную регуляризацию на веса.

Допустим, мы решили увеличить веса в $a$ раз

Так как мы шкалируем, то домножение весов $W$ на константу выходных значений слоя не меняет

$$BN((aW)u) = BN(Wu)$$

Градиент слоя по входу не меняется

$$\dfrac {\delta BN((aW)u)} {\delta u} = \dfrac {\delta BN(Wu)} {\delta u}$$

А градиент по весам уменьшается в $a$ раз

$$\dfrac {\delta BN((aW)u)} {\delta aW} = \dfrac 1 a \dfrac {\delta BN(Wu)} {\delta W} $$

Таким образом, нейросеть автоматически не дает большим весам расти

## Dropout

Одним из распространненных именно в нейросетях методом регуляризации является Dropout.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/dropout.png" width="700">

Состоит этот метод в следующем:

1. Во время обучения мы с вероятностью *p* зануляем выход нейронов слоя (например, *p* = 0.5)
2. Зануленные нейроны не участвуют в данном forward, и градиент потому к ним при backward не идет. 

3. Сила регуляризации определяется вероятностью p, чем она больше - тем сильнее регуляризация. 


### Мотивация Dropout

### Борьба с коадаптацией 

Одной из  проблем при работе с глубокими сетями является совместная адаптация нейронов. В такой сети, если все весовые коэффициенты изучаются вместе, обычно некоторые соединения будут иметь больше возможностей прогнозирования, чем другие.

Часть нейронов делает основную работу - предсказывает, а остальные могут вообще не вносить никакого вклада в итоговое предсказание. Или же другая картина - один нейрон делает кривоватое предсказание, другие его правят и в итоге первый нейрон своей ошибки не исправляет. 

Это явление называется коадаптацией. Этого нельзя было предотвратить с помощью традиционной регуляризации, такой как L1 и L2. А вот Dropout с этим хорошо борется

Отмена способности некоторых нейронов учиться во время тренировки на самом деле, оказывается, направлена на то, чтобы получить более обученные нейроны и сократить переобучение. И гарантирует, что выучиваемые индивидуальными нейронами функции хорошо работают со случайно выбранными подмножествами функций, выученных другими нейронами. 
Таким образом, Dropout приводит к появлению более требовательных нейронов, которые пытаются обойти сложные, индивидуальные особенности, которые склонны плохо обобщать, и сохраняют больше полезной информации самостоятельно. 

На следующем рисунке (извлеченном из статьи Dropout: A Simple Way to Prevent Neural Networks from Overfitting) мы находим сравнение признаков, изученных в наборе данных MNIST с одним автоэнкодером скрытого слоя, имеющим 256 выпрямленных линейных единиц без dropout (слева), и признаков, изученных той же структурой с использованием dropout в ее скрытом слое с p=0,5 (справа).

В то время как первый показывает неструктурированные, беспорядочные паттерны, которые невозможно интерпретировать, второй явно демонстрирует целенаправленное распределение веса, которое обнаруживает штрихи, края и пятна самостоятельно, нарушая их взаимозависимость с другими нейронами для выполнения этой работы.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/compare_weights_with_dropout_and_without_dropout.png" width="600">

[Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](https://jmlr.org/papers/v15/srivastava14a.html)

### Dropout как регуляризация 

Фактически, Dropout штрафует слишком сложные, неустойчивые решения. Добавляя в нейросеть Dropout, мы сообщаем ей о том, что решение, которое мы ожидаем, должно быть устойчиво к шуму

### Dropout как ансамбль 

Можно рассматривать Dropout как ансамбль нейросетей со схожими параметрами, которые мы учим одновременно, вместо того, чтобы учить каждую в отдельности, а затем результат их предсказания усредняем, [замораживая Dropout](http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html)

Фактически, возникает аналогия со случайным лесом - каждая из наших нейросетей легко выучивает выборку и переобучается - имеет низкий bias, но высокий variance. При этом за счет временного отключения активаций, каждая нейросеть видит не все объекты, а только часть. Усредняя все эти предсказания, мы уменьшаем variance. 



### Dropout помогает бороться с переобучением

Dropout, в силу указанного выше, может хорошо помогать бороться с переобучением

И в случае линейных слоев 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/dropout_solve_overfitting_problem_in_mlp_networks.png" width="500">

[Tutorial: Dropout as Regularization and Bayesian Approximation](https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/)

И в случае конволюций 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/dropout_solve_overfitting_problem_in_convolution_networks.png" width="500">

### Confidence interval от Dropout
Можно используя нейросеть с дропаутом, получить доверительный интервал для вашего предсказания. Просто не "замораживаем" dropout-слои во время предсказания, а делаем предсказания с активными dropout. 

И делаем forward через такую нейросеть для одного объекта 1000 раз. 
Сделав это 1000 раз, вы получаете распределение предсказаний, на основе которого можно делать confidence интервалы и как раз ловить те объекты, на которых нейросеть вообще не понимает, что ей делать и потому предсказывает метку или еще что-то с сильной дисперсией. 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/confidence_interval_dropout.png" width="600">

[Источник](http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html)

### Простая реализация Dropout

Pytorch

In [ ]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(3072,500),
    nn.Dropout(p=0.2),
    nn.Linear(500,1),
    )

net.train()
dummy = torch.randn((1,3072)) # batch, features
print("Train")
for i in range(5):
  print(net(dummy).item())

print("Inference")
net.eval()
for i in range(5):
  print(net(dummy).item())


In [ ]:
class BadDropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}")
        self.p = p

    def forward(self, X):
        if self.training:
            keep = torch.rand(X.size()) > self.p               
            return X * keep
        # in test time, expectation is calculated
        return X * (1 - self.p)

Нежелательное свойство представленной выше схемы состоит в том, что мы должны масштабировать активации во время прогнозирования. Поскольку производительность тестирования критически важна, лучше всего использовать инвертированный dropout, который выполняет масштабирование во время обучения. Кроме того, если мы захотим убрать dropout из кода, функция прогнозирования останется без изменений.

In [ ]:
class Dropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(
                f"Dropout probability has to be between 0 and 1, but got {p}")
        self.p = p

    def forward(self, X):
        if self.training:
            keep = torch.rand(X.size()) > self.p
            return X * keep / (1 - self.p)
        return X  # in test time, expectation is calculated intrinsically - we just not divide weights

Так как наша модель из-за Dropout ведет себя по-разному во время обучения и во время тестирования, то мы должны прямо ей сообщать, обучается она сейчас или нет. Делается это при помощи функций model.train и model.eval

### Пример борьбы с переобучением при помощи Dropout
 

Чтобы увидеть эффект, и при этом не учить нейросеть 100+ эпох, сделаем искусственный пример.

Просто добавим к линейной зависимости шум и попробуем выучить ее нейронной сетью.

[Dropout in Neural Networks with Pytorch](https://towardsdatascience.com/batch-normalization-and-dropout-in-neural-networks-explained-with-pytorch-47d7a8459bcd)

In [ ]:
import torch
N = 50  # number of data points
noise = 0.3

# generate the train data
X_train = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
Y_train = X_train + noise * torch.normal(
    torch.zeros(N, 1), # mean
    torch.ones(N, 1))  # std

# generate the test data
X_test = torch.unsqueeze(torch.linspace(-1, 1, N), 1)
Y_test = X_test + noise * torch.normal(torch.zeros(N, 1), torch.ones(N, 1))

print(f'X_train shape: {X_train.shape}\nX_test shape: {X_test.shape}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.scatter(X_train.data.numpy(), Y_train.data.numpy(),
            c='purple', alpha=0.5, label='train')
plt.scatter(X_test.data.numpy(), Y_test.data.numpy(),
            c='yellow', alpha=0.5, label='test')

X_real = np.arange(-1, 1, 0.01)
Y_real = X_real
plt.plot(X_real, Y_real, c="green", label='true')
plt.legend()
plt.show()

Модель без дропаут

In [ ]:
N_h = 100 # num of neurons
model = torch.nn.Sequential(
    torch.nn.Linear(1, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Модель с дропаут

In [ ]:
from torch import nn
N_h = 100 # num of neurons

model_dropout = nn.Sequential(
    nn.Linear(1, N_h),
    nn.Dropout(0.5),  # 50 % probability
    nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    nn.Dropout(0.2),  # 20% probability
    nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
optimizer_dropout = torch.optim.Adam(model_dropout.parameters(), lr=0.01)

In [ ]:
num_epochs = 1500
criterion = torch.nn.MSELoss()

for epoch in range(num_epochs):

    # train without dropout
    Y_pred = model(X_train)  # look at the entire data in a single shot
    loss = criterion(Y_pred, Y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # train with dropout
    Y_pred_dropout = model_dropout(X_train)
    loss_dropout = criterion(Y_pred_dropout, Y_train)
    optimizer_dropout.zero_grad()
    loss_dropout.backward()
    optimizer_dropout.step()

    if epoch % 100 == 0:

        model.eval() # not train mode
        model_dropout.eval() #  not train mode

        # get predictions
        Y_test_pred = model(X_test)
        test_loss = criterion(Y_test_pred, Y_test)

        Y_test_pred_dropout = model_dropout(X_test)
        test_loss_dropout = criterion(Y_test_pred_dropout, Y_test)
        # plotting data and predictions
        plt.scatter(X_train.data.numpy(), Y_train.data.numpy(),
                    c='purple', alpha=0.5, label='train')
        plt.scatter(X_test.data.numpy(), Y_test.data.numpy(),
                    c='yellow', alpha=0.5, label='test')
        plt.plot(X_test.data.numpy(), Y_test_pred.data.numpy(),
                 'r-', lw=3, label='normal')
        plt.plot(X_test.data.numpy(), Y_test_pred_dropout.data.numpy(),
                 'b--', lw=3,  label='dropout')

        plt.title('Epoch %d, Loss = %0.4f, Loss with dropout = %0.4f' %
                  (epoch, test_loss, test_loss_dropout))

        plt.legend()

        model.train() # train mode
        model_dropout.train() # train mode
 
        plt.pause(0.05)

Видим, что нейросеть без dropout сильно переобучилась.

## Dropconnect

Будем занулять не нейроны, а веса. Фактически - для каждого батча обрубаем часть связей между нейронами.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/dropconnect.png" width="650">

Drop Connect случайным образом отбрасывая веса, а не активации с вероятностью *p*.

DropConnect похож на Dropout, поскольку он вводит динамическую разреженность в модель, но отличается тем, что разреженность зависит от весов *W*, а не от выходных векторов слоя. Другими словами, полностью связанный слой с DropConnect становится разреженно связанным слоем, в котором соединения выбираются случайным образом на этапе обучения. 

В принципе, вариантов зануления чего-то в нейросетке можно предложить великое множество, в разных ситуациях будут работать разные ([в этом списке](https://paperswithcode.com/methods/category/regularization)  много Drop...)

# Оптимизация весов нейросетей

Методов тоже много, расскажем о популярных ([неполный список](https://paperswithcode.com/methods/category/stochastic-optimization))

Эти методы реализованы в модуле torch.optim
При этом, что важно, подсчет градиентов лосс функции по весам никак не зависит от оптимизаторов из пакета. На прошлых занятиях мы с вами видели, что pytorch вычисляет градиенты автоматически, на основе оптимизаций.


## Обзор популярных оптимизаторов

### SGD
Обычный стохастичный градиентный спуск. Вы его уже реализовывали. 
Просто обновляем веса в соответствии с текущем градиентом по ним, домножая градиент на постоянный коэффициент $\eta$

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/stochastic_gradient_descent.gif" width="950">

```
grad = evaluate_gradient(loss_fun,x,weights)
weights = weights - learning_rate * grad
```

До сих пор является достаточно популярным методом обучения нейросетей, ибо простой, не требует дополнительных параметров кроме $\eta$, и сам по себе обычно дает не плохие результаты. 

Если же важна каждая сотая в качестве, то нужно либо использовать его вкупе с другими технологиями (о них - ниже), либо использовать другие способы. 

In [ ]:
from torch import optim

parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], lr=0.001)

Минусы SGD:

 1. Если функция ошибки быстро меняется в одном направлении и медленно - в другом, то это приводит к резким изменениям направления  градиентов и замедляет процесс обучения


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/stohastic_gradient_descent_no_momentum.gif" width="500">


[Machine Learning Optimization Methods “Mechanics, Pros and Cons”](https://salmenzouari.medium.com/machine-learning-optimization-methods-mechanics-pros-and-cons-81b720194292)

 2. Может застревать в локальных минимумах или седловых точках (точках, где все производные равны 0, но не являющихся минимума/ максимумами). В них градиент равен 0, веса не обновляются - конец оптимизации. 

Пример таких точек:


Точка 0 у функции $x^3$, не имеющей минимума или максимума вовсе

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/getting_stuck_in_local_minimum_example.png" width="400">


Или точка 0, 0 у функции z = $x^2 - y^2$


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/saddle_point_example.png" width="400">


[Седловая точка
](https://ru.wikipedia.org/wiki/%D0%A1%D0%B5%D0%B4%D0%BB%D0%BE%D0%B2%D0%B0%D1%8F_%D1%82%D0%BE%D1%87%D0%BA%D0%B0)

 3. Так как мы оцениваем градиенты по малой части выборки, они могут плохо отображать градиент по всей выборке и являться шумными. В результате часть шагов градиентного спуска делаются впустую или во вред. 
 
 4. Мы применяем один и тот же learning rate ко всем параметрам, что не всегда разумно. Параметр, отвечающий редкому классу, будет обучаться медленнее остальных. 
 
 5. Просто медленнее сходится

### Momentum

Чтобы избежать проблем 1-3, можно использовать momentum - фактически, мы добавляем нашему движению инерции. Если представить наши текущие веса как координаты шарика, и мы этот шарик пытаемся загнать в наиболее глубокую дырку, то у шарика теперь появился вес.

[Градиентный спуск, как учатся нейронные сети(Видео)](https://youtu.be/IHZwWFHWa-w)

Сначала пытаемся поменять направление движения шарика с прежнего направления с учетом текущего градиента
$$v_{t+1} = \rho v_t + \nabla_wL(x, y, W)$$

Вычисляем, куда он покатится
$$w_{t+1} = w_t - \eta v_{t+1}$$

```
vx = 0 # velocity
while True:
  dx = evaluate_gradient(loss_fun,x,weights)
  vx = vx*rho + dx # rho 0.9 .. 0.99 'friction'
  x -= lr *dx
```

In [ ]:
class SGD_with_momentum:  
  def __init__(self,weights,lr = 0.001):
    self.vx = torch.zeros_like(weights) # vx = 0
    self.weights = weights
    self.lr = lr
  
  def step(self):
    dx = self.weights.grad
    self.vx = self.vx * 0.9 + dx # 0.9 .. 0.99 'friction'
    self.weights -= self.lr * self.vx 


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/advantages_wtih_momentum.png" width="480">

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], momentum=0.9, lr=0.001)

Теперь мы быстрее достигаем локального минимума и можем выкатываться из совсем неглубоких. Градиентн стал менее подвержен шуму, меньше асциллирует

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/stohastic_gradient_descent_no_momentum.gif" width="400">

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/stohastic_gradient_descent_with_momentum.gif" width="400">

У этого подхода есть одна опасность - мы можем выкатиться за пределы минимума, к которому стремимся, а потом какое-то время к нему возвращаться. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/problem_of_big_momentum_value.gif" width="700">

[optimizer-visualization](https://github.com/Jaewan-Yun/optimizer-visualization)

Чтобы с этим бороться, предложен другой способ подсчета момента инерции

### NAG (Nesterov momentum)

Будем сначала смещаться в сторону, куда привел бы нас наш накопленный градиент, там считать новый градиент и смещаться по нему. 
В результате перескоки через минимум будут менее значительными, и мы будем быстрее сходиться

$$v_{t+1} = \rho v_t +  \nabla_w L(w + \rho v_t )$$

$$w_{t} = w_{t-1} - \eta v_{t} $$


<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/nesterov_momentum.png" width="800">

На практике эту формулу все равно можно записать так, чтобы задача вычисления градиента ложилась на сам оптимизатор
(к примеру, [реализация в PyTorch](https://github.com/pytorch/pytorch/blob/master/torch/optim/_functional.py))

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], momentum=0.9, nesterov=True, lr=0.001)

### Adaptive Learning Rate

Оба озвученных улучшения никак не борются с тем, что некоторые параметры нашей нейросети могут получать информацию для обновления реже, чем остальные. Например, фильтр, отличающий сиамских кошек от остальных, будет работать только тогда, когда в выборке попадутся конкретно сиамские кошки. А их может быть и не очень много, по батчам мы их не размажем (либо батчей получится очень много, если нарочно балансировать)

Другая ситуация - учим нейросеть на словах из русского языка. Есть веса, отвечающие за редкие слова, например "молвить". Вы часто в языке встречаете слово молвить? Молвят устами, а они тоже встречаются не часто. В результате, если learning rate постоянен, то мы выучим параметры для слова молвить плохо. 

Единственный путь - завести для каждого параметра индивидуальный learning rate.

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/stohastic_gradient_descent_no_momentum.gif" width="500">

### Adagrad

Будем хранить для каждого параметра сумму квадратов его градиентов. 

И будем добавлять наш градиент с коэффициентом, обратно пропорциональным корню из этой суммы (фактически - некая аппроксимация средней амплитуды градиента нашего параметра). 

$$ w = w - \eta \frac{l}{\sqrt{G} + e} \odot (\nabla_w L(x,y,W)) $$

$$ G = \sum_{t=1}^T \nabla_w L(x,y,w_t)^2 $$



In [ ]:
class AdaGrad:  
  def __init__(self,weights, lr = 0.001):
    self.grad_squared = torch.zeros_like(weights) # tensor
    self.weights = weights
  
  def step(self):
    dx = self.weights.grad
    self.grad_squared += dx*dx
    self.weights -= self.lr * dx / (torch.sqrt(self.grad_squared) ++ 1e-7)

В результате, если градиент у нашего веса часто большой - коэффициент будет уменьшаться. 

Единственная проблема - при такой формуле наш learning rate неминуемо в конце концов затухает (так как сумма квадратов не убывает)


In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adagrad([parameters])

### RMSprop

Давайте устроим "забывание" предыдущих квадратов градиентов. Просто будем домножать их на некий коэффициент меньше 1


$$v_t = \alpha v_{t-1} + (1-\alpha) (\nabla_w L(x,y,w_t))^2$$

$$w = w - \frac{\eta}{\sqrt{v_t }+ e} \odot \nabla_w L(x,y,W)$$

In [ ]:
class RMSprop():  
  def __init__(self,weights, lr = 0.001):
    self.grad_squared = torch.zeros_like(weights) # tensor
    self.weights = weights
    self.decay_rate = 0.99 
  
  def step(self):
    dx = self.weights.grad
    self.grad_squared = dx*dx(1-self.decay_rate) + self.decay_rate*self.grad_squared
    self.weights -= self.lr * dx / (torch.sqrt(self.grad_squared) + 1e-7)

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.RMSprop([parameters], alpha=0.99)

### Adam

Одним из самых популярных адаптивных оптимизаторов является Adam. 
Получается он за счет объединения идеи с инерцией и идеи с суммой квадратов. 

$$ m_t = \beta_1 m_{t-1} + (1-\beta_1) (\nabla_w L(x,y,w_t)) $$
$$ v_t = \beta_2 v_{t-1} + (1-\beta_2) (\nabla_w L(x,y,w_t)^2) $$
$$ w = w - \eta \cdot \frac{m_t}{\sqrt{v_t} + e} $$

In [ ]:
class ADAM:  
  def __init__(self,weights, lr = 0.001):
    self.weights = weights
    self.velocity = torch.zeros_like(weights) # ~ vx = 0
    self.grad_squared = torch.zeros_like(weights) # tensor    
    self.decay_rate = 0.99 
    self.friction = 0.9
  
  def step(self):
    dx = self.weights.grad
    self.vx = self.velocity * self.friction +  (1-self.friction)*dx # first momentum
    self.grad_squared = self.grad_squared  + (1-self.decay_rate)*dx*dx # second momentum
    self.weights -= self.lr * velocity / (torch.sqrt(self.grad_squared) + 1e-7)

Чтобы в начале у нас получались очень большие шаги, будем дополнительно модицифировать инерцию и сумму квадратов

$$ m_t = \frac{m_t}{1-\beta_1^t} $$

$$ v_t = \frac{v_t}{1-\beta_2^t} $$

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adam([parameters], betas=(0.9, 0.999))

#### L2-loss

Все оптимизаторы так же поддерживают возможность добавления к ним напрямую L2-loss, коэффициент перед этим лоссом -  $\textrm{weight_decay}$

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.RMSprop([parameters], alpha=0.99, weight_decay=0.001)

Нюанс в том, что в Adam L2-loss учитывается не совсем верно. Потому есть поправленная версия Adam - AdamW. Но не факт, что она всегда лучше работает

## Сравнение оптимизаторов 

У каждого из предложенных оптимизаторов есть минусы и плюсы 



### Методы с инерцией сходятся к решению более плавно, но могут "перелетать"

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/convergence_optimizers.gif" width="500">



### Методы с адаптивным learning rate быстрее сходятся, более стабильны и меньше случайно блуждают

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_with_adaptive_learning_rate.gif" width="500">


###  Алгоритмы без адаптивного learning rate сложнее выбираются из локальных минимумом

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_without_adaptive_learning_rate.gif" width="450">

### Алгоритмы с инерцией осцилируют в седловых точках прежде чем найти верный путь

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/methods_with_momentum_in_saddle_point.gif" width="450">

<p><em>Иллюстрации <a href="https://imgur.com/a/Hqolp">Alec Radford</a></p> </em>


# Режимы обучения

Нам не обязательно поддерживать один и тот же learning rate в течение всего обучения. Более того, для того же SGD есть гарантии, что если правильно подобрать схему уменьшения learning rate, он сойдется к глобальному оптимуму.


Мы можем менять learning rate по некоторым правилам.  



## Ранняя остановка

Можем использовать критерий ранней остановки - когда лосс на валидационной выборке не улучшается какое-то количество эпох(patience), умножаем learning rate на некое значение(factor)

<img src ="https://edunet.kea.su/repo/EduNet-content/L07/img_license/early_stopping.png" width="500">

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 'min',
                                                 factor=0.1,
                                                 patience=5)

Применим к нашей модели

(выполнение занимает ~ 5 минут)

## Понижение шага обучения на каждой эпохе 

Домножать learning rate на gamma каждую эпоху

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size=2,
                                            gamma=0.1)

## Neural Network WarmUp

Также, для достаточно больших нейронных сетей практикуют следующую схему (**gradual warmup**, [изначальная статья](https://arxiv.org/pdf/1706.02677.pdf)):

Поставить изначальный learning rate значительно ниже того, с которого мы обычно начинаем обучение. За несколько эпох, например, 5, довести learning rate от этого значения до требуемого. За счет этого нейросеть лучше "адаптируется" к нашим данным. 

Также такой learning schedule позволяет адаптивным оптимизаторам лучше оценить значения learning rate для разных параметров

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L07/neural_network_warmup.png" width="1000">


kn на картинке - это размер одного батча

### Взаимодействие learning schedule и адаптивного изменения learning rate

И то, и другое меняет learning rate: learning scheduler - глобально, а адаптивные оптимизаторы - для каждого веса отдельно 

Часто их применяют вместе, особенно в случае критерия ранней остановки и WarmUp

<font size = "6"/> Ссылки:</font>

[A journey into Optimization algorithms for Deep Neural Networks](https://theaisummer.com/optimization/)

[Optimizers Explained - Adam, Momentum and Stochastic Gradient Descent](https://mlfromscratch.com/optimizers-explained/#adam)

[Батч-нормализация. In-layer normalization techniques for training very deep neural networks](https://theaisummer.com/normalization/)

[Циклический learning rate](https://towardsdatascience.com/adaptive-and-cyclical-learning-rates-using-pytorch-2bf904d18dee)

[Разные функции активации, затухающие и взрывающщиеся градиенты и т.д](https://mlfromscratch.com/activation-functions-explained/#/)

[Визуализация разных оптимизаторов в ipynb, но на tensorflow](https://nbviewer.jupyter.org/github/ilguyi/optimizers.numpy/blob/master/optimizer.tf.all.opt.plot.ipynb)